In [141]:
import numpy as np
import scipy.spatial.distance

In [142]:
grid = np.array([[-1,0],[0,-1], [2,2], [3,3]])
ref_esp = np.array([-1,-1,1, 1])
mol_atom = np.array([[0,0],[1,0],[0,1]])
q_atom_init = np.array([-2,1,1])
mol_vsite = np.array([[0,0],[1,0],[0,1],[1,1]])
q_vsite_init = np.array([-2,1,1,0])

In [376]:
grid = np.array([[1,2],[2,1], [2,3], [3,2]])
ref_esp = np.array([1,1,1,1])
mol_atom = np.array([[0,0],[1,0],[0,1]])
q_atom_init = np.array([-2,1,1])
mol_vsite = np.array([[0,0],[1,0],[0,1],[2,2]])
q_vsite_init = np.array([1,1,1,0])

T = np.array([
    [-1,0,0],
    [0,-1,0],
    [0,0,-1],
    [1,1,1],
])
n_cc = T.shape[1]
p = np.zeros((n_cc,1))

In [348]:
grid = np.array([[-3],[-1], [29999], [30001]])
ref_esp = np.array([2,2,2,2])
mol_atom = np.array([[-2]])
q_atom_init = np.array([4])
mol_vsite = np.array([[-2],[30000]])
q_vsite_init = np.array([4,0])

T = np.array([
    [-1],
    [1],
])
n_cc = T.shape[1]
p = np.zeros((n_cc,1))

In [377]:
def distance(A, B):
    return np.linalg.norm(A-B)

def epot(grid, ptls, q):
    v = np.zeros(grid.shape[0])
    for l, g in enumerate(grid):
        for j, ptl in enumerate(ptls):
            v[l] += q[j]/distance(g, ptl)
    return v  

def objective(ref_esp, train_esp):
    return np.sum((ref_esp-train_esp)**2)

In [408]:
v_calc = epot(grid, mol_atom, q_atom_init); v_calc

array([0.31267959, 0.31267959, 0.11508096, 0.11508096])

In [409]:
v_corr = epot(grid, mol_vsite, np.dot(T, p)); v_corr

array([0., 0., 0., 0.])

In [410]:
v_diff = ref_esp - epot(grid, mol_vsite, q_vsite_init)
print(ref_esp)
print(v_diff)

[1 1 1 1]
[-0.65432038 -0.65432038  0.05286875  0.05286875]


In [411]:
objective(ref_esp, v_calc)

2.5109821048387166

In [412]:
R = 1.0 / scipy.spatial.distance.cdist(grid, mol_vsite)
R

array([[0.4472136 , 0.5       , 0.70710678, 1.        ],
       [0.4472136 , 0.70710678, 0.5       , 1.        ],
       [0.2773501 , 0.31622777, 0.35355339, 1.        ],
       [0.2773501 , 0.35355339, 0.31622777, 1.        ]])

In [413]:
A = np.dot(np.dot(T.T, np.dot(R.T, R)), T); A

array([[1.65559138, 1.39958311, 1.39958311],
       [1.39958311, 1.22122412, 1.1769377 ],
       [1.39958311, 1.1769377 , 1.22122412]])

In [414]:
A = T.T @ R.T @ R @ T ; A

array([[1.65559138, 1.39958311, 1.39958311],
       [1.39958311, 1.22122412, 1.1769377 ],
       [1.39958311, 1.1769377 , 1.22122412]])

In [387]:
B = np.dot(T.T, np.dot(R.T, v_diff)); B

array([-0.64698763, -0.44847919, -0.44847919])

In [388]:
B = T.T @ R.T @ v_diff ; B

array([-0.64698763, -0.44847919, -0.44847919])

In [389]:
help(scipy.optimize.least_squares)

Help on function least_squares in module scipy.optimize._lsq.least_squares:

least_squares(fun, x0, jac='2-point', bounds=(-inf, inf), method='trf', ftol=1e-08, xtol=1e-08, gtol=1e-08, x_scale=1.0, loss='linear', f_scale=1.0, diff_step=None, tr_solver=None, tr_options={}, jac_sparsity=None, max_nfev=None, verbose=0, args=(), kwargs={})
    Solve a nonlinear least-squares problem with bounds on the variables.
    
    Given the residuals f(x) (an m-D real function of n real
    variables) and the loss function rho(s) (a scalar function), `least_squares`
    finds a local minimum of the cost function F(x)::
    
        minimize F(x) = 0.5 * sum(rho(f_i(x)**2), i = 0, ..., m - 1)
        subject to lb <= x <= ub
    
    The purpose of the loss function rho(s) is to reduce the influence of
    outliers on the solution.
    
    Parameters
    ----------
    fun : callable
        Function which computes the vector of residuals, with the signature
        ``fun(x, *args, **kwargs)``, i.e.

In [390]:
A.shape

(3, 3)

In [391]:
B.shape

(3,)

In [392]:
import scipy.optimize
import numpy as np

def fun(x, A, b):
    return ((np.dot(A, x.reshape(-1, 1)) - b.reshape(-1,1)) ** 2).flat

x0 = np.zeros((n_cc,))
ret = scipy.optimize.least_squares(
    fun,
    x0,
    args=(A, B),
    verbose=2,
    max_nfev=300,
    method='lm'
)

`xtol` termination condition is satisfied.
Function evaluations 109, initial cost 1.2806e-01, final cost 2.5352e-29, first-order optimality 2.37e-21.


In [401]:
ret.x

array([-5.61964684,  3.09265354,  3.09265354])

In [402]:
q_train = q_vsite_init + np.dot(T,ret.x); q_train

array([ 6.61964684, -2.09265354, -2.09265354,  0.56566024])

In [403]:
v_corr_train = epot(grid, mol_vsite, np.dot(T, ret.x)) ; v_corr_train

array([-0.65432035, -0.65432035,  0.05286878,  0.05286878])

In [404]:
v_calc

array([0.31267959, 0.31267959, 0.11508096, 0.11508096])

In [405]:
v_train = epot(grid, mol_vsite, q_train); v_train

array([1.00000002, 1.00000002, 1.00000003, 1.00000003])

In [406]:
ref_esp

array([1, 1, 1, 1])

In [407]:

print("INIT:", objective(ref_esp, v_calc))
print("FIT: ", objective(ref_esp, v_train))

INIT: 2.5109821048387166
FIT:  3.0417921027515986e-15
